In [1]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.merge import add

from keras import regularizers

import numpy as np

Using TensorFlow backend.


# Model Definition

## Encoder
The role of this model is to create feature vector from raw images.

Borrow from https://github.com/uzh-rpg/rpg_public_dronet

Note:
    Layer naming convention: `layer_name(str) + stage(int) + block(str)`

### Define a convolutional block

In [2]:
def convolutional_block(X, num_filters, shape_filters, strides, stage):
    """
    Implementation of convolutional block in Residual network
    
    Input:
        X (tensor): input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
        num_filters (list of 3 ints): list of number of filters
        shape_filters (list of 3 ints): list of filters' shape
        strides (list of 3 ints): list of strides
        stage (int): stage of this convolutional block in the whole ResNet
        
    Output:
        tensor of shape (m, n_H, n_W, n_C)
    """
    
    # retrieve filters shape from filters
    n1, n2, n3 = num_filters
    f1, f2, f3 = shape_filters
    
    # retrieve strides from strides
    s1, s2, s3 = strides
    
    # create name
    bn_name_base = 'bn_' + str(stage) + '_'
    conv_name_base = 'conv_' + str(stage) + '_'
    
    # save value of X
    X_shorcut = X
    
    # First component of the main path
    X = keras.layers.normalization.BatchNormalization(name=bn_name_base + 'a')(X)
    X = Activation('relu')(X)
    X = Conv2D(n1, (f1, f1), strides=[s1, s1], padding='same',
               name=conv_name_base + 'a',
               kernel_initializer='he_normal',
               kernel_regularizer=regularizers.l2(1e-4))(X)
    
    # Second component of the main path
    X = keras.layers.normalization.BatchNormalization(name=bn_name_base + 'b')(X)
    X = Activation('relu')(X)
    X = Conv2D(n2, (f2, f2), strides=[s2, s2], padding='same',
               name=conv_name_base + 'b',
               kernel_initializer='he_normal',
               kernel_regularizer=regularizers.l2(1e-4))(X)
    
    # Short-cut
    X_shorcut = Conv2D(n3, (f3, f3), strides=[s3, s3], padding='same', name=conv_name_base + 'c')(X_shorcut)
    
    X = add([X, X_shorcut])
    
    return X


### Assemble 3 convolutional block to make a ResNet 

In [3]:
def resnet8_clean(input_shape):
    """
    Define encoder architecture as ResNet8
    
    Input:
        input_shape (list of ints): shape of input image [n_H, n_W, n_C]
        
    Output:
        model: a Model instance
    """
    
    # Input
    X_input = Input(shape=input_shape)
    
    # Apply 1st convolution & max pooling on input
    X = Conv2D(32, (5, 5), strides=[2,2], padding='same', name='conv_0')(X_input)
    X = MaxPooling2D(pool_size=(3, 3), strides=[2,2])(X) 
    
    # First convolutional block
    X = convolutional_block(X, [32, 32, 32], [3, 3, 1], [2, 1, 2], stage=1)
    
    # Second convolutional block
    X = convolutional_block(X, [64, 64, 64], [3, 3, 1], [2, 1, 2], stage=2)
    
    # Third convolutional block
    X = convolutional_block(X, [128, 128, 128], [3, 3, 1], [2, 1, 2], stage=3)
    
    # Output layer
    X = Flatten()(X)
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)
    
    # Define model
    model = Model(inputs=[X_input], outputs=[X])
    print(model.summary())
    
    return model

### Create model & load weights

In [4]:
encoder = resnet8_clean([200, 200, 1])
encoder.load_weights('./model/named_resnet8_best_weights.h5', by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__

In [16]:
names = [weight.name for layer in encoder.layers for weight in layer.weights]
weights = encoder.get_weights()

names

['conv_0/kernel:0',
 'conv_0/bias:0',
 'bn_1_a/gamma:0',
 'bn_1_a/beta:0',
 'bn_1_a/moving_mean:0',
 'bn_1_a/moving_variance:0',
 'conv_1_a/kernel:0',
 'conv_1_a/bias:0',
 'bn_1_b/gamma:0',
 'bn_1_b/beta:0',
 'bn_1_b/moving_mean:0',
 'bn_1_b/moving_variance:0',
 'conv_1_b/kernel:0',
 'conv_1_b/bias:0',
 'conv_1_c/kernel:0',
 'conv_1_c/bias:0',
 'bn_2_a/gamma:0',
 'bn_2_a/beta:0',
 'bn_2_a/moving_mean:0',
 'bn_2_a/moving_variance:0',
 'conv_2_a/kernel:0',
 'conv_2_a/bias:0',
 'bn_2_b/gamma:0',
 'bn_2_b/beta:0',
 'bn_2_b/moving_mean:0',
 'bn_2_b/moving_variance:0',
 'conv_2_b/kernel:0',
 'conv_2_b/bias:0',
 'conv_2_c/kernel:0',
 'conv_2_c/bias:0',
 'bn_3_a/gamma:0',
 'bn_3_a/beta:0',
 'bn_3_a/moving_mean:0',
 'bn_3_a/moving_variance:0',
 'conv_3_a/kernel:0',
 'conv_3_a/bias:0',
 'bn_3_b/gamma:0',
 'bn_3_b/beta:0',
 'bn_3_b/moving_mean:0',
 'bn_3_b/moving_variance:0',
 'conv_3_b/kernel:0',
 'conv_3_b/bias:0',
 'conv_3_c/kernel:0',
 'conv_3_c/bias:0']

In [26]:
conv2d_1_a_kernel = weights[14]
print(conv2d_1_a_kernel.shape)
conv2d_1_a_kernel

(1, 1, 32, 32)


array([[[[ 0.03906486,  0.23744462,  0.35682636, ..., -0.15168019,
          -0.09894565,  0.10449842],
         [ 0.08876824, -0.04708504, -0.5501143 , ...,  0.35634387,
           0.98491377,  0.17439887],
         [ 0.08548965, -0.03150511, -0.2497131 , ...,  0.32054433,
           0.6079249 , -0.26140153],
         ...,
         [-0.17021403, -0.07588131,  0.01981616, ..., -0.26841697,
           0.4311797 , -0.16715825],
         [ 0.10458679, -0.24335292, -0.13646314, ..., -0.168561  ,
          -0.06483505, -0.07373277],
         [ 0.18375537, -0.17555152, -0.13670585, ...,  0.08006119,
           0.29586527,  0.01597549]]]], dtype=float32)

## Decoder

This model decode the feature vector created by the Encoder to produce the prediction for the `PLANNING_HORIZON` (in form of classes of steering angles) 

In [7]:
import h5py

In [8]:
w1 = h5py.File('./model/best_weights.h5')
allKeys = w1.keys()
list(allKeys)

['activation_1',
 'activation_2',
 'activation_3',
 'activation_4',
 'activation_5',
 'activation_6',
 'activation_7',
 'activation_8',
 'add_1',
 'add_2',
 'add_3',
 'batch_normalization_1',
 'batch_normalization_2',
 'batch_normalization_3',
 'batch_normalization_4',
 'batch_normalization_5',
 'batch_normalization_6',
 'conv2d_1',
 'conv2d_10',
 'conv2d_2',
 'conv2d_3',
 'conv2d_4',
 'conv2d_5',
 'conv2d_6',
 'conv2d_7',
 'conv2d_8',
 'conv2d_9',
 'dense_1',
 'dense_2',
 'dropout_1',
 'flatten_1',
 'input_1',
 'max_pooling2d_1']

In [27]:
origin_conv2d_2_wts = w1['conv2d_4']['conv2d_4']['kernel:0']
origin_conv2d_2_kernel = origin_conv2d_2_wts.value
print(origin_conv2d_2_kernel.shape)
print(origin_conv2d_2_kernel)

(1, 1, 32, 32)
[[[[ 0.03906486  0.23744462  0.35682636 ... -0.15168019 -0.09894565
     0.10449842]
   [ 0.08876824 -0.04708504 -0.5501143  ...  0.35634387  0.98491377
     0.17439887]
   [ 0.08548965 -0.03150511 -0.2497131  ...  0.32054433  0.6079249
    -0.26140153]
   ...
   [-0.17021403 -0.07588131  0.01981616 ... -0.26841697  0.4311797
    -0.16715825]
   [ 0.10458679 -0.24335292 -0.13646314 ... -0.168561   -0.06483505
    -0.07373277]
   [ 0.18375537 -0.17555152 -0.13670585 ...  0.08006119  0.29586527
     0.01597549]]]]


In [28]:
diff = conv2d_1_a_kernel.squeeze() - origin_conv2d_2_kernel.squeeze() 
np.sum(np.square(diff))

0.0